In [ ]:
!pip install pyspark==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 28 kB/s 
     |████████████████████████████████| 198 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6b196c20f7f4b6e6e7813357733bc252de0365bc4fd262f3ed7f9428f3e7b839
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import  SparkSession

In [ ]:
ss = SparkSession.builder.getOrCreate()

In [ ]:
!nohup ssh -o StrictHostKeyChecking=no -R mars.ru77.ru:XXXX:*:4040 aig@mars.ru77.ru -p 2222 &

nohup: appending output to 'nohup.out'


In [ ]:
! wget "http://files.grouplens.org/datasets/movielens/ml-latest.zip" -O "movielens.zip"

--2022-06-19 07:03:04--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘movielens.zip’

movielens.zip       100%[===================>] 264.28M  95.3MB/s    in 2.8s    

2022-06-19 07:03:07 (95.3 MB/s) - ‘movielens.zip’ saved [277113433/277113433]



In [ ]:
! unzip movielens.zip -d movielens/

Archive:  movielens.zip
   creating: movielens/ml-latest/
  inflating: movielens/ml-latest/links.csv  
  inflating: movielens/ml-latest/tags.csv  
  inflating: movielens/ml-latest/genome-tags.csv  
  inflating: movielens/ml-latest/ratings.csv  
  inflating: movielens/ml-latest/README.txt  
  inflating: movielens/ml-latest/genome-scores.csv  
  inflating: movielens/ml-latest/movies.csv  


In [ ]:
! cat movielens/ml-latest/README.txt

Summary

This dataset (ml-latest) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 27753444 ratings and 1108997 tag applications across 58098 movies. These data were created by 283228 users between January 09, 1995 and September 26, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 1 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `genome-scores.csv`, `genome-tags.csv`, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLe

In [ ]:
ratings = ss.read.csv('movielens/ml-latest/ratings.csv', header=True, inferSchema=True)

In [ ]:
ratings.schema

StructType(List(StructField(userId,IntegerType,true),StructField(movieId,IntegerType,true),StructField(rating,DoubleType,true),StructField(timestamp,IntegerType,true)))

In [ ]:
def show(df):
  return df.limit(5).toPandas()

In [ ]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
from pyspark.sql import functions as sf

In [ ]:
movie_mean_rating = ratings.groupby('movieId').agg(sf.mean('rating'))

In [ ]:
show(movie_mean_rating)

,movieId,avg(rating)
0,1591,2.646666
1,1645,3.535294
2,471,3.652909
3,1088,3.248014
4,1580,3.578533


In [ ]:
movies = ss.read.csv('movielens/ml-latest/movies.csv', header=True, inferSchema=True)

In [ ]:
show(movies)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies = movies.join(movie_mean_rating, on='movieId', how='inner')

In [ ]:
movies.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [movieId#58, title#59, genres#60, avg(rating)#29]
   +- BroadcastHashJoin [movieId#58], [movieId#17], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#127]
      :  +- Filter isnotnull(movieId#58)
      :     +- FileScan csv [movieId#58,title#59,genres#60] Batched: false, DataFilters: [isnotnull(movieId#58)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/movielens/ml-latest/movies.csv], PartitionFilters: [], PushedFilters: [IsNotNull(movieId)], ReadSchema: struct<movieId:int,title:string,genres:string>
      +- HashAggregate(keys=[movieId#17], functions=[avg(rating#18)])
         +- Exchange hashpartitioning(movieId#17, 200), ENSURE_REQUIREMENTS, [id=#124]
            +- HashAggregate(keys=[movieId#17], functions=[partial_avg(rating#18)])
               +- Filter isnotnull(movieId#17)
                  +- FileScan csv

In [ ]:
movies.cache()

DataFrame[movieId: int, title: string, genres: string, avg(rating): double]

In [ ]:
movies.cache??

In [ ]:
movies.persist??

In [ ]:
movies.cache().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [movieId#58, title#59, genres#60, avg(rating)#29]
   +- BroadcastHashJoin [movieId#58], [movieId#17], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#127]
      :  +- Filter isnotnull(movieId#58)
      :     +- FileScan csv [movieId#58,title#59,genres#60] Batched: false, DataFilters: [isnotnull(movieId#58)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/movielens/ml-latest/movies.csv], PartitionFilters: [], PushedFilters: [IsNotNull(movieId)], ReadSchema: struct<movieId:int,title:string,genres:string>
      +- HashAggregate(keys=[movieId#17], functions=[avg(rating#18)])
         +- Exchange hashpartitioning(movieId#17, 200), ENSURE_REQUIREMENTS, [id=#124]
            +- HashAggregate(keys=[movieId#17], functions=[partial_avg(rating#18)])
               +- Filter isnotnull(movieId#17)
                  +- FileScan csv

In [ ]:
ss.sparkContext.setCheckpointDir('./chkp')

In [ ]:
movies = movies.checkpoint()

In [ ]:
ls -al .

total 270648
drwxr-xr-x 1 root root      4096 Jun 19 07:04 ./
drwxr-xr-x 1 root root      4096 Jun 19 07:01 ../
drwxr-xr-x 3 root root      4096 Jun 19 07:04 chkp/
drwxr-xr-x 4 root root      4096 Jun 15 13:41 .config/
drwxr-xr-x 3 root root      4096 Jun 19 07:03 movielens/
-rw-r--r-- 1 root root 277113433 Dec  3  2019 movielens.zip
-rw------- 1 root root        63 Jun 19 07:03 nohup.out
drwxr-xr-x 1 root root      4096 Jun 15 13:42 sample_data/


In [ ]:
ls -al ./chkp/

total 12
drwxr-xr-x 3 root root 4096 Jun 19 07:04 ./
drwxr-xr-x 1 root root 4096 Jun 19 07:04 ../
drwxr-xr-x 3 root root 4096 Jun 19 07:05 005929b3-bb68-4d42-8d96-f1e8a99d7c4c/


In [ ]:
show(movies)

,movieId,title,genres,avg(rating)
0,1591,Spawn (1997),Action|Adventure|Sci-Fi|Thriller,2.646666
1,1645,The Devil's Advocate (1997),Drama|Mystery|Thriller,3.535294
2,471,"Hudsucker Proxy, The (1994)",Comedy,3.652909
3,1088,Dirty Dancing (1987),Drama|Musical|Romance,3.248014
4,1580,Men in Black (a.k.a. MIB) (1997),Action|Comedy|Sci-Fi,3.578533


In [ ]:
@sf.udf
def title(t):
  return '('.join(t.split('(')[:-1])

In [ ]:
movies = movies.withColumn('short_title', title('title'))

In [ ]:
show(movies.select(sf.split('genres', '\|')))

,"split(genres, \|, -1)"
0,"[Action, Adventure, Sci-Fi, Thriller]"
1,"[Drama, Mystery, Thriller]"
2,[Comedy]
3,"[Drama, Musical, Romance]"
4,"[Action, Comedy, Sci-Fi]"


In [ ]:
movies.select(sf.split('genres', '\|')).printSchema()

root
 |-- split(genres, \|, -1): array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
col = sf.explode(sf.split('genres', '\|'))
genres = movies.select(col.alias('genre'))
genres = list(genres.drop_duplicates().toPandas()['genre'])

In [ ]:
for g in genres:
    movies = movies.withColumn(g, sf.col('genres').like('%' + g + '%'))

In [ ]:
movies = movies.drop('genres')

In [ ]:
show(movies)

,movieId,title,avg(rating),short_title,Crime,Romance,Thriller,Adventure,Drama,War,...,Animation,Film-Noir,(no genres listed),IMAX,Horror,Western,Comedy,Children,Action,Sci-Fi
0,1591,Spawn (1997),2.646666,Spawn,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,True,True
1,1645,The Devil's Advocate (1997),3.535294,The Devil's Advocate,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,471,"Hudsucker Proxy, The (1994)",3.652909,"Hudsucker Proxy, The",False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,1088,Dirty Dancing (1987),3.248014,Dirty Dancing,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,1580,Men in Black (a.k.a. MIB) (1997),3.578533,Men in Black (a.k.a. MIB),False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,True


In [ ]:
@sf.udf
def year(title):
  t = title[-5:-1]
  try:
    int(t)
  except:
    t = -1
  return t

In [ ]:
movies = movies.withColumn('year', year('title'))

In [ ]:
movies = movies.withColumn('year', sf.col('year').cast('int'))

In [ ]:
from pyspark.sql.window import Window

In [ ]:
gs_col = sf.array([sf.col(g) for g in genres]).alias('gs')

In [ ]:
ws = Window.partitionBy('gs').orderBy('year').rangeBetween(-2, 0)

In [ ]:
show(movies.select('movieId', 'title', 'year', gs_col)\
  .withColumn('releases_before_colunt', sf.count('movieId').over(ws) - 1) \
  .filter(sf.col('year') != -1).orderBy('year'))

,movieId,title,year,gs,releases_before_colunt
0,159257,6A,6,"[False, False, False, False, False, False, Fal...",0
1,175335,The Garden of Afflictions 2017,201,"[False, False, False, False, False, False, Fal...",1
2,172451,Irwin & Fran 2013,201,"[False, False, False, False, False, False, Fal...",1
3,148054,Passage de Venus (1874),1874,"[False, False, False, False, False, False, Tru...",0
4,148048,Sallie Gardner at a Gallop (1878),1878,"[False, False, False, False, False, False, Fal...",0


In [ ]:
movies.select('movieId', 'title', 'year','avg(rating)' , gs_col)\
  .withColumn('releases_before_colunt', sf.max('avg(rating)').over(ws) - sf.col('avg(rating)')).limit(40).toPandas()

,movieId,title,year,avg(rating),gs,releases_before_colunt
0,168278,Science Fiction Volume One: The Osiris Child,-1,2.774194,"[False, False, False, False, False, False, Fal...",1.120806
1,40697,Babylon 5,-1,3.895000,"[False, False, False, False, False, False, Fal...",0.000000
2,103774,Sol (2012),-1,2.500000,"[False, False, False, False, False, False, Fal...",1.395000
3,125978,Santa Claus (1898),1898,2.500000,"[False, False, False, False, False, False, Fal...",0.000000
4,105776,"Trip to Mars, A (1910)",1910,2.857143,"[False, False, False, False, False, False, Fal...",0.000000
5,181261,Charleston Parade (1927),1927,3.500000,"[False, False, False, False, False, False, Fal...",0.000000
6,144828,The Tunnel (1933),1933,3.000000,"[False, False, False, False, False, False, Fal...",0.000000
7,186911,Deluge (1933),1933,2.750000,"[False, False, False, False, False, False, Fal...",0.250000
8,126436,Cosmic Journey (1935),1935,3.250000,"[False, False, False, False, False, False, Fal...",0.000000
9,7128,Things to Come (1936),1936,3.497512,"[False, False, False, False, False, False, Fal...",0.000000


In [ ]:
ws = Window.partitionBy('gs').orderBy('year').rangeBetween(-5, 5)

In [ ]:
sc = ss.sparkContext

In [ ]:
# поиск количество слова 'devil' в title
import re

In [ ]:
devil_re = sc.broadcast(re.compile('devil', re.IGNORECASE))

In [ ]:
acc = sc.accumulator(0)

In [ ]:
@sf.udf
def devil(title):
  if devil_re.value.search(title):
    acc.add(1)
  return title

In [ ]:
md = movies.withColumn('title2', devil('title'))

In [ ]:
acc.value

0

In [ ]:
md.collect()

In [ ]:
acc.value

183